In [1]:
import pandas as pd, requests, pymongo, psycopg2, numpy as np
import pandas.io.sql as sqlio

pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)

# Traffic Collision Data

In [2]:
response = requests.get("https://data.lacity.org/api/views/yru6-6re4/rows.json?accessType=DOWNLOAD")

In [3]:
all_data =response.json()
cols = [col_name["name"] for col_name in all_data["meta"]["view"]["columns"]]


In [4]:
values=list(map(list, zip(*[data for data in all_data["data"]])))

In [5]:
keyvalues =[dict(zip(cols, item)) for item in zip(*values)]

In [57]:
df=pd.DataFrame(keyvalues)

In [58]:
df.head()

,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,Report ID,Report Type,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Disposition Description,Address,Cross Street,LAT,LON,Location,Booking Date,Booking Time,Booking Location,Booking Location Code,Zip Codes,Census Tracts,Precinct Boundaries,LA Specific Plans,Council Districts,Neighborhood Councils (Certified)
0,row-3a4a~pj2u-dqj3,00000000-0000-0000-C90F-5CEA34B80D10,0,1598834264,None,1598834608,None,{ },121920046,RFC,2012-09-28T00:00:00,0930,19,Mission,1998,60,M,H,None,None,M,25620(A)BP,None,MISDEMEANOR COMPLAINT FILED,7600 WOODMAN AV,None,34.2111,-118.4309,POINT (-118.4309 34.2111),None,None,None,None,19730,151,462,None,3,59
1,row-85wz.haaj.h3n4,00000000-0000-0000-FA1C-4B9B04D2A76D,0,1598834264,None,1601485878,None,{ },090712341,RFC,2011-03-10T00:00:00,0940,07,Wilshire,0776,45,M,B,18,Drunkeness,M,41.27CLAMC,DRINKING IN PUBLIC,MISDEMEANOR COMPLAINT FILED,4500 W WASHINGTON BL,None,34.0399,-118.3375,POINT (-118.3375 34.0399),None,None,None,None,23080,651,1106,None,12,69
2,row-auh4-u3kb_76qe,00000000-0000-0000-D672-57BE81D6CDBA,0,1598834264,None,1598834608,None,{ },121909585,RFC,2012-04-01T00:00:00,1715,19,Mission,1993,37,M,H,None,None,M,25620(A)BP,None,MISDEMEANOR COMPLAINT FILED,8100 SEPULVEDA PL,None,34.2208,-118.4662,POINT (-118.4662 34.2208),None,None,None,None,19730,141,424,None,3,59
3,row-z4kz~dahc.yv84,00000000-0000-0000-E705-26F5FBB5BA45,0,1598834264,None,1601485878,None,{ },100109543,RFC,2010-04-14T00:00:00,1510,01,Central,0158,38,F,W,13,Prostitution/Allied,M,653.22(A)PC,LOITER:INTENT:PROSTITUTION,MISDEMEANOR COMPLAINT FILED,7TH,TOWNE AV,34.0395,-118.2405,POINT (-118.2405 34.0395),None,None,None,None,23074,537,1018,None,9,76
4,row-bxgf_ve7e.xt6j,00000000-0000-0000-4F13-CB3BFAAEA9C7,0,1598834264,None,1601485878,None,{ },100208368,RFC,2010-03-18T00:00:00,2110,02,Rampart,0275,40,F,W,24,Miscellaneous Other Violations,M,103.107.1BL,ESCORT WITHOUT PERMIT,MISDEMEANOR COMPLAINT FILED,1900 W OLYMPIC BL,None,34.0508,-118.2777,POINT (-118.2777 34.0508),None,None,None,None,22723,573,628,None,11,68


In [59]:
df.shape

(1322821, 39)

In [60]:
df['Age'].isna().sum()
df['Age'].fillna(0,inplace=True)

In [61]:
df['Age']=pd.to_numeric(df['Age'])
n = int(sum(df['Age'])/len(df['Age']))
n

34

In [62]:
df['Age'].replace(0,34,inplace=True)

In [63]:
l=[]
for i in df['Age']:
    if i<19:
        l.append('0-18')
    elif i<31:
        l.append('19-30')
    elif i<46:
        l.append('31-45')
    elif i<61:
        l.append('46-60')
    else:
        l.append('60+')
df['Age group']=l

In [64]:
df.columns

Index(['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at',
       'updated_meta', 'meta', 'Report ID', 'Report Type', 'Arrest Date',
       'Time', 'Area ID', 'Area Name', 'Reporting District', 'Age', 'Sex Code',
       'Descent Code', 'Charge Group Code', 'Charge Group Description',
       'Arrest Type Code', 'Charge', 'Charge Description',
       'Disposition Description', 'Address', 'Cross Street', 'LAT', 'LON',
       'Location', 'Booking Date', 'Booking Time', 'Booking Location',
       'Booking Location Code', 'Zip Codes', 'Census Tracts',
       'Precinct Boundaries', 'LA Specific Plans', 'Council Districts',
       'Neighborhood Councils (Certified)', 'Age group'],
      dtype='object')

In [65]:
df.drop(['sid','id','position','created_meta','updated_at','updated_meta','meta',
         'Report ID','Area ID','Area Name','Reporting District','Charge','Address', 'Cross Street', 'LAT', 'LON',
       'Location', 'Booking Date', 'Booking Time', 'Booking Location',
       'Booking Location Code', 'Zip Codes', 'Census Tracts',
       'Precinct Boundaries', 'LA Specific Plans', 'Council Districts',
       'Neighborhood Councils (Certified)','Charge Group Code'],axis=1,inplace=True)


In [66]:
# df.drop(['created_at'],axis=1,inplace=True)

In [18]:
df.drop(['created_at'],axis=1,inplace=True)

In [67]:
df['Arrest Type Code'].value_counts()

M    805283
F    421719
I     56105
O     28523
D     11190
1         1
Name: Arrest Type Code, dtype: int64

In [68]:
df['Disposition Description'].value_counts()

MISDEMEANOR COMPLAINT FILED         828494
FELONY COMPLAINT FILED              205464
DISTRICT ATTORNEY REJECT             43497
COUNSELED/RELEASED                   38579
849(B)(1) PC/TECHNICAL ARREST        32507
CITY ATTORNEY REJECT                 30361
OTHER (REQUIRES ADDITION ACTION)     23967
DEPARTMENT OF SOCIAL SERVICES         4273
JUVENILE TRAFFIC-MISDEMEANOR          3348
PROBATION                             1798
OTHER DISPOSITION                     1700
OTHER LAW ENFORCEMENT AGENCY          1673
PRE-FILING INVESTIGATION               117
RELEASED/INSUFFICENT EVIDENCE           82
CALIFORNIA YOUTH AUTHORITY              56
PROVED ADULT                            55
ACTION SUSP                             40
COMMUNITY SERVICE                       31
EXONERATED/INNOCENT                     23
NON-BOOK PETITION                        9
RELEASED PETITION                        8
FIRE DEPARTMENT                          5
RELEASED TO MILITARY AUTHORITY           4
DEPARTMENT 

In [80]:
df['Disposition Description'].isna().sum()

0

In [20]:
df['Charge Description'].value_counts()

DRUNK DRIVING ALCOHOL/DRUGS                 98634
DRINKING IN PUBLIC                          93916
CORPORAL INJURY ON SPOUSE/COHABITANT/ETC    47228
POSSESSION CONTROLLED SUBSTANCE             45462
FTA AFTER WRITTEN PROMISE                   31639
                                            ...  
FRAUD USE PHONE/TELEGRAPH SERVICE               1
BARBERING OR COSMETOLOGY W/O A LICENSE          1
PANDER IN A HOUSE OF PROSTITUTION               1
FAIL TO USE CHILD RESTRAINT SYSTEM              1
PREPARE FALSE DOCUMENTARY EVIDENCE              1
Name: Charge Description, Length: 2393, dtype: int64

In [21]:

df['Charge Group Description'].value_counts()

Miscellaneous Other Violations    247497
Narcotic Drug Laws                165025
Driving Under Influence           121542
Drunkeness                        116190
Aggravated Assault                 86924
Larceny                            70161
Other Assaults                     63812
Moving Traffic Violations          53693
Liquor Laws                        52482
Prostitution/Allied                40994
Disorderly Conduct                 37394
Weapon (carry/poss)                31354
Robbery                            26851
Vehicle Theft                      25354
Burglary                           24473
Sex (except rape/prst)             13479
Pre-Delinquency                    11898
Fraud/Embezzlement                 11450
Against Family/Child                7464
Forgery/Counterfeit                 7005
Non-Criminal Detention              6909
Receive Stolen Property             6732
Disturbing the Peace                3150
Rape                                3022
Gambling        

In [70]:
for i in df.columns:
    try:
        df[i]=pd.to_numeric(df[i])
    except:
        pass

In [71]:
df.head()

,created_at,Report Type,Arrest Date,Time,Age,Sex Code,Descent Code,Charge Group Description,Arrest Type Code,Charge Description,Disposition Description,Age group
0,1598834264,RFC,2012-09-28T00:00:00,930.0,60,M,H,None,M,None,MISDEMEANOR COMPLAINT FILED,46-60
1,1598834264,RFC,2011-03-10T00:00:00,940.0,45,M,B,Drunkeness,M,DRINKING IN PUBLIC,MISDEMEANOR COMPLAINT FILED,31-45
2,1598834264,RFC,2012-04-01T00:00:00,1715.0,37,M,H,None,M,None,MISDEMEANOR COMPLAINT FILED,31-45
3,1598834264,RFC,2010-04-14T00:00:00,1510.0,38,F,W,Prostitution/Allied,M,LOITER:INTENT:PROSTITUTION,MISDEMEANOR COMPLAINT FILED,31-45
4,1598834264,RFC,2010-03-18T00:00:00,2110.0,40,F,W,Miscellaneous Other Violations,M,ESCORT WITHOUT PERMIT,MISDEMEANOR COMPLAINT FILED,31-45


In [72]:
df.drop(['Charge Description'],axis=1,inplace=True)

In [73]:
df['Arrest Date']=pd.to_datetime(df['Arrest Date'])

In [74]:
df1=pd.get_dummies(df['Sex Code'])
df['Male']=df1['M']
df['Female']=df1['F']

In [75]:
df1=pd.get_dummies(df['Descent Code'])
df['vd_B']=df1['B']
df['vd_W']=df1['W']
df['vd_A']=df1['A']
df['vd_H']=df1['H']
df['vd_O']=df1['O']

In [81]:
df1=pd.get_dummies(df['Age group'])
df['0-18']=df1['0-18']
df['19-30']=df1['19-30']
df['31-45']=df1['31-45']
df['46=60']=df1['46-60']
df['60+']=df1['60+']

In [82]:
df['Disposition Description'].isna().sum()
df['Disposition Description'].fillna('FELONY COMPLAINT FILED',inplace=True)



In [83]:
df['MISDEMEANOR'] = list(map(lambda x: 1 if x=='MISDEMEANOR COMPLAINT FILED' else 0,df['Disposition Description']))
df['FELONY'] = list(map(lambda x: 0 if x=='MISDEMEANOR COMPLAINT FILED' else 1,df['Disposition Description']))

In [84]:
df1=pd.get_dummies(df['Charge Group Description'])
df['Drugs_related']=df1['Narcotic Drug Laws']
df['Drunkeness']=df1['Drunkeness']+df1['Driving Under Influence']
df['Assault']=df1['Aggravated Assault']+df1['Other Assaults']
df['Traffic_Violations']=df1['Moving Traffic Violations']
df['Larceny']=df1['Larceny']


In [85]:
df['Arrest Type Code'].value_counts()

M    805283
F    421719
I     56105
O     28523
D     11190
1         1
Name: Arrest Type Code, dtype: int64

In [86]:
df.head()

,created_at,Report Type,Arrest Date,Time,Age,Sex Code,Descent Code,Charge Group Description,Arrest Type Code,Disposition Description,Age group,Male,Female,vd_B,vd_W,vd_A,vd_H,vd_O,0-18,19-30,31-45,46=60,60+,MISDEMEANOR,FELONY,Drugs_related,Drunkeness,Assault,Traffic_Violations,Larceny
0,1598834264,RFC,2012-09-28,930.0,60,M,H,None,M,MISDEMEANOR COMPLAINT FILED,46-60,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0
1,1598834264,RFC,2011-03-10,940.0,45,M,B,Drunkeness,M,MISDEMEANOR COMPLAINT FILED,31-45,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0
2,1598834264,RFC,2012-04-01,1715.0,37,M,H,None,M,MISDEMEANOR COMPLAINT FILED,31-45,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0
3,1598834264,RFC,2010-04-14,1510.0,38,F,W,Prostitution/Allied,M,MISDEMEANOR COMPLAINT FILED,31-45,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
4,1598834264,RFC,2010-03-18,2110.0,40,F,W,Miscellaneous Other Violations,M,MISDEMEANOR COMPLAINT FILED,31-45,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0


In [87]:
df.columns

Index(['created_at', 'Report Type', 'Arrest Date', 'Time', 'Age', 'Sex Code',
       'Descent Code', 'Charge Group Description', 'Arrest Type Code',
       'Disposition Description', 'Age group', 'Male', 'Female', 'vd_B',
       'vd_W', 'vd_A', 'vd_H', 'vd_O', '0-18', '19-30', '31-45', '46=60',
       '60+', 'MISDEMEANOR', 'FELONY', 'Drugs_related', 'Drunkeness',
       'Assault', 'Traffic_Violations', 'Larceny'],
      dtype='object')

In [88]:
df.drop(['Report Type','Age','Sex Code','Descent Code','Arrest Type Code'],axis=1,inplace=True)

In [89]:
# df['Time']=pd.to_numeric(df['Time'])
# n = int(sum(df['Age'])/len(df['Age']))
pd.Series.mode(df['Time'])

0    1600.0
dtype: float64

In [90]:
df['Time'].fillna(1600,inplace=True)
df['hr']=df['Time']/100
df['hr'].astype(int)
l=[]
for hr in df['hr']:
    if hr<5:
        l.append("night")
    elif hr<12:
        l.append('morning')
    elif hr<17:
        l.append('afternoon')
    elif hr<20:
        l.append('evening')
    else:
        l.append("night")
df['timeofday']=l
df1=pd.get_dummies(df['timeofday'])
df['night']=df1['night']
df['morning']=df1['morning']
df['afternoon']=df1['afternoon']
df['evening']=df1['evening']
df.drop(['timeofday','hr'],axis=1,inplace=True)

In [91]:
df.drop(['Charge Group Description'],axis=1,inplace=True)
df.head()

,created_at,Arrest Date,Time,Disposition Description,Age group,Male,Female,vd_B,vd_W,vd_A,vd_H,vd_O,0-18,19-30,31-45,46=60,60+,MISDEMEANOR,FELONY,Drugs_related,Drunkeness,Assault,Traffic_Violations,Larceny,night,morning,afternoon,evening
0,1598834264,2012-09-28,930.0,MISDEMEANOR COMPLAINT FILED,46-60,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
1,1598834264,2011-03-10,940.0,MISDEMEANOR COMPLAINT FILED,31-45,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0
2,1598834264,2012-04-01,1715.0,MISDEMEANOR COMPLAINT FILED,31-45,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
3,1598834264,2010-04-14,1510.0,MISDEMEANOR COMPLAINT FILED,31-45,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
4,1598834264,2010-03-18,2110.0,MISDEMEANOR COMPLAINT FILED,31-45,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0


In [92]:
df.drop(['Time','Age group'],axis=1,inplace=True)
df.head()

,created_at,Arrest Date,Disposition Description,Male,Female,vd_B,vd_W,vd_A,vd_H,vd_O,0-18,19-30,31-45,46=60,60+,MISDEMEANOR,FELONY,Drugs_related,Drunkeness,Assault,Traffic_Violations,Larceny,night,morning,afternoon,evening
0,1598834264,2012-09-28,MISDEMEANOR COMPLAINT FILED,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
1,1598834264,2011-03-10,MISDEMEANOR COMPLAINT FILED,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0
2,1598834264,2012-04-01,MISDEMEANOR COMPLAINT FILED,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
3,1598834264,2010-04-14,MISDEMEANOR COMPLAINT FILED,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
4,1598834264,2010-03-18,MISDEMEANOR COMPLAINT FILED,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0


In [96]:
df['count']=1

In [97]:
sum(df['count'])

1322821

In [98]:
dff=df.groupby('Arrest Date', as_index=False).agg({'count':sum,'Male':sum,'Female':sum,
                                                     'vd_B':sum,'vd_W':sum,'vd_A':sum,'vd_H':sum,'vd_O':sum,'0-18':sum,
                                                     '19-30':sum,'31-45':sum,'46=60':sum, '60+':sum,
                                                   'MISDEMEANOR':sum, 'FELONY':sum, 'Drugs_related':sum, 'Drunkeness':sum,
                                                   'Assault':sum, 'Traffic_Violations':sum,
                                                   'Larceny':sum,
                                                   'night':sum, 'morning':sum, 'afternoon':sum, 'evening':sum,})

In [100]:
dff.sort_values('Arrest Date')

,Arrest Date,count,Male,Female,vd_B,vd_W,vd_A,vd_H,vd_O,0-18,19-30,31-45,46=60,60+,MISDEMEANOR,FELONY,Drugs_related,Drunkeness,Assault,Traffic_Violations,Larceny,night,morning,afternoon,evening
0,2010-01-01,381,323.0,58,88.0,73,1,199.0,19,51,165.0,110,48,7,208,173,40,106,71,21,9,207.0,45,87,42
1,2010-01-02,364,294.0,70,105.0,58,1,177.0,21,43,130.0,107,77,7,236,128,48,57,33,14,18,140.0,56,108,60
2,2010-01-03,308,259.0,49,83.0,53,2,151.0,18,65,102.0,78,52,11,175,133,37,56,35,15,25,141.0,40,83,44
3,2010-01-04,342,268.0,74,117.0,57,0,164.0,4,58,110.0,89,74,11,173,169,69,64,38,15,14,108.0,70,111,53
4,2010-01-05,412,322.0,90,129.0,79,1,185.0,15,56,129.0,127,93,7,243,169,78,77,32,23,18,118.0,73,129,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2019-12-27,242,191.0,51,73.0,51,1,107.0,10,8,101.0,80,42,11,142,100,33,29,42,8,13,90.0,33,75,44
3648,2019-12-28,219,171.0,48,58.0,37,2,112.0,10,8,100.0,74,35,2,134,85,21,40,37,5,10,82.0,47,54,36
3649,2019-12-29,209,172.0,37,65.0,32,1,99.0,12,6,82.0,76,32,13,124,85,15,35,54,7,14,81.0,56,50,22
3650,2019-12-30,204,151.0,53,47.0,39,0,109.0,9,8,97.0,62,30,7,123,81,18,27,36,5,15,72.0,57,55,20


In [104]:
dff.to_csv("arrest.csv",index=False)

In [199]:
dff=df.groupby('Date Occurred', as_index=False).agg({'hr': np.mean,'Victim Age':np.median,'Premise Description':pd.Series.mode,
'Male':sum,'Female':sum,'vd_B':sum,'vd_W':sum,'vd_A':sum,'vd_H':sum,'vd_O':sum,'0-18':sum,'19-30':sum,'31-45':sum,'46=60':sum, '60+':sum,})

In [200]:
dff.head()

,Date Occurred,hr,Victim Age,Premise Description,Male,Female,vd_B,vd_W,vd_A,vd_H,vd_O,0-18,19-30,31-45,46=60,60+
0,2010-01-01,10.824180,35.0,STREET,77,41,16,30,6,45,13,0,37,55,21,9
1,2010-01-02,12.988228,35.0,STREET,52,26,6,27,4,30,11,0,16,39,17,7
2,2010-01-03,11.679697,35.0,STREET,63,35,13,26,2,36,15,0,25,38,22,14
3,2010-01-04,13.941961,37.5,STREET,67,33,14,31,5,35,14,0,24,45,24,9
4,2010-01-05,14.216607,38.0,STREET,72,40,18,32,4,37,14,0,26,45,25,16


In [201]:
dff.shape

(4370, 16)

In [203]:
dff.to_csv("traffic_collision.csv",index=False)

In [204]:
ddf = pd.read_csv("traffic_collision.csv")

In [ ]:
def create_table(df,creat_table_query):

    try:
        dbConnection = psycopg2.connect(
            user = "dap",
            password = "dap",
            host = "127.0.0.1",
            port = "5432",
            database = "dap")
        dbConnection.set_isolation_level(0) # AUTOCOMMIT
        dbCursor = dbConnection.cursor()
        qr = creat_table_query
   
        dbCursor.execute(qr)
        dbCursor.close()
    except (Exception , psycopg2.Error) as dbError :
        print ("Error while connecting to PostgreSQL step 2", dbError)
    finally:
        if dbConnection in locals(): 
            dbConnection.close()
            
def insert_data(df,insert_into_query):
    try:
        dbConnection = psycopg2.connect(
            user = "dap",
            password = "dap",
            host = "127.0.0.1",
            port = "5432",
            database = "dap")
        dbConnection.set_isolation_level(0) # AUTOCOMMIT
        dbCursor = dbConnection.cursor()   

        df.replace('Suppressed (counts 1-9)', np.NaN, inplace=True)
        df.replace('None', np.NaN, inplace=True) 
        df.fillna(0,inplace=True)
        tpls = [tuple(x) for x in df.to_numpy()]
        sql32 = insert_into_query

        for row in tpls:
            print("Data inserted using execute_many() successfully...")

            dbCursor.execute(sql32.format(*row))

        dbConnection.commit()
        dbCursor.close()
    except (Exception , psycopg2.Error) as dbError :
        print ("Error:", dbError)
        print(row)
    finally:
        if dbConnection in locals(): 
            dbConnection.close()

In [ ]:
ct = '''CREATE TABLE arrest_final(

ar_arrest_date date ,
ar_count float,
ar_male float,
ar_female float,

ar_person_race_black float,
ar_person_race_white float,
ar_person_race_asian float,
ar_person_race_hispanic float,
ar_person_race_other float,

ar_age_0_18 float,
ar_age_19_30 float,
ar_age_31_45 float,
ar_age_46_60 float,
ar_age_61abv float,

ar_MISDEMEANOR float,
ar_FELONY float,

ar_reason_Drugs_related float,
ar_reason_Drunkeness float,
ar_reason_Assault float,
ar_reason_Traffic_Violations float,
ar_reason_Larceny float,

ar_time_night float,
ar_time_morning float,
ar_time_afternoon float,
ar_time_evening float

);
'''

it =  '''INSERT INTO arrest_final(

ar_arrest_date,
ar_count,
ar_male,
ar_female,

ar_person_race_black,
ar_person_race_white,
ar_person_race_asian,
ar_person_race_hispanic,
ar_person_race_other,

ar_age_0_18,
ar_age_19_30,
ar_age_31_45,
ar_age_46_60,
ar_age_61abv,

ar_MISDEMEANOR,
ar_FELONY,

ar_reason_Drugs_related,
ar_reason_Drunkeness,
ar_reason_Assault,
ar_reason_Traffic_Violations,
ar_reason_Larceny,

ar_time_night,
ar_time_morning,
ar_time_afternoon,
ar_time_evening

)
VALUES('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}',
'{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}');'''

create_table(ar,ct)
insert_data(ar,it)